In [2]:
import numpy as np
import cv2
import os
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import math
import open3d as o3d
import plotly.graph_objects as go
import matplotlib.pyplot as plt


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
def draw_geometries(geometries):
    graph_objects = []

    for geometry in geometries:
        geometry_type = geometry.get_geometry_type()
        
        if geometry_type == o3d.geometry.Geometry.Type.PointCloud:
            points = np.asarray(geometry.points)
            colors = None
            if geometry.has_colors():
                colors = np.asarray(geometry.colors)
            elif geometry.has_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.normals) * 0.5
            else:
                geometry.paint_uniform_color((1.0, 0.0, 0.0))
                colors = np.asarray(geometry.colors)

            scatter_3d = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=1, color=colors))
            graph_objects.append(scatter_3d)

        if geometry_type == o3d.geometry.Geometry.Type.TriangleMesh:
            triangles = np.asarray(geometry.triangles)
            vertices = np.asarray(geometry.vertices)
            colors = None
            if geometry.has_triangle_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.triangle_normals) * 0.5
                colors = tuple(map(tuple, colors))
            else:
                colors = (1.0, 0.0, 0.0)
            
            mesh_3d = go.Mesh3d(x=vertices[:,0], y=vertices[:,1], z=vertices[:,2], i=triangles[:,0], j=triangles[:,1], k=triangles[:,2], facecolor=colors, opacity=0.50)
            graph_objects.append(mesh_3d)
        
    fig = go.Figure(
        data=graph_objects,
        layout=dict(
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False)
            )
        )
    )
    fig.show()
o3d.visualization.draw_geometries = draw_geometries # replace function

In [5]:
def calculate_normal_vector(points):
    pca = PCA(n_components=3)
    pca.fit(points)
    normal_vector = pca.components_[-1]  # The normal vector is the eigenvector with the smallest eigenvalue

    normal_vector = np.array(normal_vector)
    angle_with_x_axis = np.arccos(np.dot(normal_vector, [1, 0, 0]))
    angle_with_y_axis = np.arccos(np.dot(normal_vector, [0, 1, 0]))
    angle_with_z_axis = np.arccos(np.dot(normal_vector, [0, 0, 1]))
    normal_vector = [angle_with_x_axis, angle_with_y_axis, angle_with_z_axis]
    # print(normal_vector)
    return normal_vector

def angle_with_z_axis(vector):
    vector = vector / np.linalg.norm(vector)
    return np.arccos(np.dot(vector, [0, 0, -1]))

In [6]:
def xy2radius(x, y):
    return math.sqrt(x**2 + y**2)

def xy2theta(x, y):
    return math.atan2(y, x)

In [7]:
def apply_gabor_filter(image, ksize=13, sigma=1, theta=0, lambd=7, gamma=0.5):
    kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma)
    return cv2.filter2D(image, cv2.CV_32F, kernel)

def cluster_images(images, n_clusters):
    flattened_images = [img.flatten() for img in images]
    kmeans = KMeans(n_clusters=n_clusters).fit(flattened_images)
    return kmeans.labels_, kmeans.cluster_centers_

In [8]:
class Zone:
    def __init__(self, pcd_file_path):
        self.pcd_file_path = pcd_file_path
        self.point_cloud = None
        self.load_point_cloud()

    def calculate_normal_vector(self, points):
        pca = PCA(n_components=3)
        pca.fit(points)
        normal_vector = pca.components_[-1]

        return normal_vector

    def angle_with_z_axis(self, vector):
        vector = vector / np.linalg.norm(vector)
        return np.arccos(np.dot(vector, [0, 0, -1]))

    def apply_gabor_filter(self, image, ksize=13, sigma=1, theta=0, lambd=7, gamma=0.5):
        kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma)
        return cv2.filter2D(image, cv2.CV_32F, kernel)


    def count_stripes(self, image, theta):
        filtered_image = self.apply_gabor_filter(image, ksize=13, sigma=1, theta=theta, lambd=7, gamma=0.5)
        local_maxima = (filtered_image == cv2.dilate(filtered_image, np.ones((3,3))))
        return np.sum(local_maxima)

    def segment_czm(cloud, min_range_, max_range_, min_range_z2_, min_range_z3_, min_range_z4_, num_zones=4, num_sectors_each_zone=[72, 48, 48, 64], num_rings_each_zone=[4, 4, 5, 5], layer_colors=[0,1,2,3]):
        ring_sizes_ = [(min_range_z2_ - min_range_) / num_rings_each_zone[0],
                    (min_range_z3_ - min_range_z2_) / num_rings_each_zone[1],
                    (min_range_z4_ - min_range_z3_) / num_rings_each_zone[2],
                    (max_range_ - min_range_z4_) / num_rings_each_zone[3]]

        sector_sizes_ = [2 * math.pi / num_sectors_each_zone[0], 
                        2 * math.pi / num_sectors_each_zone[1], 
                        2 * math.pi / num_sectors_each_zone[2], 
                        2 * math.pi / num_sectors_each_zone[3]]

        czm = [[
            [Zone() for _ in range(num_sectors_each_zone[zone_idx])]
            for _ in range(num_rings_each_zone[zone_idx])
        ] for zone_idx in range(num_zones)]

        for pt in cloud:
            x, y, z, intensity = pt
            r = xy2radius(x, y)
            if r <= max_range_ and r > min_range_:
                theta = xy2theta(x, y)
                zone_idx = ring_idx = sector_idx = 0

                if r < min_range_z2_:
                    zone_idx = 0
                    ring_idx = min(int((r - min_range_) / ring_sizes_[0]), num_rings_each_zone[0]-1)
                    sector_idx = min(int(theta / sector_sizes_[0]), num_sectors_each_zone[0]-1)
                    czm[zone_idx][ring_idx][sector_idx].points.append([x, y, z, intensity, layer_colors[0]])
                elif r < min_range_z3_:
                    zone_idx = 1
                    ring_idx = min(int((r - min_range_z2_) / ring_sizes_[1]), num_rings_each_zone[1]-1)
                    sector_idx = min(int(theta / sector_sizes_[1]), num_sectors_each_zone[1]-1)
                    czm[zone_idx][ring_idx][sector_idx].points.append([x, y, z, intensity, layer_colors[1]])
                elif r < min_range_z4_:
                    zone_idx = 2
                    ring_idx = min(int((r - min_range_z3_) / ring_sizes_[2]), num_rings_each_zone[2]-1)
                    sector_idx = min(int(theta / sector_sizes_[2]), num_sectors_each_zone[2]-1)
                    czm[zone_idx][ring_idx][sector_idx].points.append([x, y, z, intensity, layer_colors[2]])
                else:
                    zone_idx = 3
                    ring_idx = min(int((r - min_range_z4_) / ring_sizes_[3]), num_rings_each_zone[3]-1)
                    sector_idx = min(int(theta / sector_sizes_[3]), num_sectors_each_zone[3]-1)
                    czm[zone_idx][ring_idx][sector_idx].points.append([x, y, z, intensity, layer_colors[3]])

        return czm

    def process_sectors(self, sectors, img_dim=15, MIN_POINTS=10):
        images = []

        for sector in sectors:
            if len(sector.points) > MIN_POINTS:
                points = np.array([pt[:3] for pt in sector.points])
                normal_vector = self.calculate_normal_vector(points)

                if self.angle_with_z_axis(normal_vector) >= 1.1 and self.angle_with_z_axis(normal_vector) <= 1.9:
                    image = np.zeros((img_dim, img_dim), dtype=np.uint8)

                    # Add the logic to normalize and convert the sector to an image
                    # ...

                    images.append(image)

        return images

    def analyze_cluster(self, cluster_path):
        image_files = [os.path.join(cluster_path, f) for f in os.listdir(cluster_path) if os.path.isfile(os.path.join(cluster_path, f))]
        images = [cv2.imread(f, cv2.IMREAD_GRAYSCALE) for f in image_files]
        thetas = [np.pi * i / 4 for i in range(8)]
        stripe_counts = [np.mean([self.count_stripes(img, theta=t) for t in thetas]) for img in images]
        return np.mean(stripe_counts)


    def visualize_clusters(self, images, n_clusters, base_output_folder):
        # Flatten images for clustering
        flattened_images = [img.flatten() for img in images]
        kmeans = KMeans(n_clusters=n_clusters).fit(flattened_images)

        # Save images to clusters
        for i, label in enumerate(kmeans.labels_):
            cluster_path = os.path.join(base_output_folder, f"cluster_{label}")
            if not os.path.exists(cluster_path):
                os.makedirs(cluster_path)
            cv2.imwrite(os.path.join(cluster_path, f"image_{i}.png"), images[i])

        # Analyze clusters
        cluster_stripe_counts = {}
        for i in range(n_clusters):
            cluster_path = os.path.join(base_output_folder, f"cluster_{i}")
            cluster_stripe_counts[i] = self.analyze_cluster(cluster_path)

        # Sort clusters by mean stripe count
        sorted_clusters = sorted(cluster_stripe_counts.items(), key=lambda x: x[1])

        # Visualize clusters
        # ...

        print("Clusters have been visualized by stripe pattern repetition.")



In [ ]:
pcd_file_path = "/media/rtlink/JetsonSSD-256/Download/Rellis_3D_os1_cloud_node_color_ply/Rellis-3D/00000/os1_cloud_node_color_ply/pcd_files/frame002846-1581624937_362.pcd"
czm = CZM(pcd_file_path)
sectors = czm.segment_czm()
images = czm.process_sectors(sectors)
czm.visualize_clusters(images, n_clusters=5, base_output_folder='output_clusters')
